In [ ]:
# !pip install -q -r requirements.txt
from __reading import *

In [ ]:
CORPUS_DATA = None
def get_corpus_data(path_corpus=PATH_CORPUS):
    def without_full_text(d):
        return {k: v for k, v in d.items() if k != 'fullText'}

    global CORPUS_DATA
    if CORPUS_DATA is None:
        CORPUS_DATA = pd.DataFrame(
            tqdm(
                (without_full_text(d) for d in orjsonl.stream(path_corpus)),
                total=CORPUS_NUM_SENTS
            )
        ).set_index('url')
    return CORPUS_DATA

In [ ]:
df_corpus = get_corpus_data()
df_instances = get_instances_data()
word2data = get_word2data()

In [ ]:
total_df = df_instances.merge(df_corpus, on="url", how="left").fillna("")
total_df['year'] = pd.to_numeric(total_df.publicationYear, errors="coerce")
total_df['decade'] = total_df.year.astype(int) // 10 * 10
total_df['prev_pos'] = total_df.token0.map(lambda x: word2data[x]['pos'] if x in word2data else None).fillna("")
total_df['next_pos'] = total_df.token2.map(lambda x: word2data[x]['pos'] if x in word2data else None).fillna("")
total_df_adjs = total_df[total_df.prev_pos.str.startswith("j")]

In [492]:
total_df_adjs.year.max()

np.int64(2018)

In [ ]:
word

In [508]:
len(total_df_adjs)

25762

In [515]:
total_df_adjs.isPartOf.value_counts().describe()

count       7.000000
mean     3680.285714
std      2766.820661
min      1496.000000
25%      1780.000000
50%      3138.000000
75%      4059.000000
max      9450.000000
Name: count, dtype: float64

In [517]:
len(total_df_adjs)

25762

In [524]:
ok_words = {
    "distant",
    "correct",
    "careful",
    "political",
    "feminist",
    "nuanced",
    "close",
}
df_adjs_sample = total_df_adjs.query('year>=1920 & token0 in @ok_words').sort_values('year').groupby(['decade','token0']).sample(10,replace=True).drop_duplicates(subset=['url','sent','token0'])
df_adjs_sample

,url,page_num,sent_num,sent,context0,context1,context2,token_num,token0,token1,...,title,volumeNumber,wordCount,tdmCategory,abstract,subTitle,year,decade,prev_pos,next_pos
13646,http://www.jstor.org/stable/3714692,2,2,Indeed it is evident that Miss Butler's statem...,"is a relief, also, to find a manual which is n...",Indeed it is evident that Miss Butler's statem...,"If I venture on a few criticisms of detail, fo...",21,careful,reading,...,Review Article,19,2129,[Arts - Literature],,,1924,1920,jj,ii
55204,http://www.jstor.org/stable/3714637,8,2,"Awl, but careful __reading__ of its examples, ...",264 Notes on Old and Middle English by the N.E...,"Awl, but careful reading of its examples, of t...","Chaucer, C.T., B, 782ff.\n(Man of Lawes Tale)....",3,careful,reading,...,Notes on Old and Middle English,22,3403,[Linguistics - Grammar],,,1927,1920,jj,ii
52091,http://www.jstor.org/stable/3714868,1,11,The whole forms an imposing edifice based on a...,and 355 pp.\n42s.\nIn appraising a book writte...,The whole forms an imposing edifice based on a...,The Bibliography alone occupies 11 pp.\nand re...,13,careful,reading,...,Review Article,22,2110,[Linguistics - Language],,,1927,1920,jj,cc
15095,http://www.jstor.org/stable/457250,17,9,"On careful __reading__, neither Drayton's refe...","Jonson's conversations with Drummond, which gi...","On careful reading, neither Drayton's referenc...",The passage on Jonson in Of Poets and Poesie e...,2,careful,reading,...,Drayton's Sirena,39,8632,[Arts - Literature],,,1924,1920,jj,rr
68000,http://www.jstor.org/stable/457713,8,13,Moreover as time went on they tended to grow l...,Probably she was right.\nThe letters are an ef...,Moreover as time went on they tended to grow l...,But on this visit to Dunlop House Burns had co...,32,careful,reading,...,New Light on the Burns-Dunlop Estrangement,44,4489,[Arts - Literature],,,1929,1920,jj,ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,http://www.jstor.org/stable/10.1086/674691,1,37,The merits of Politicizing Domesticity are not...,Knoppers’s assertion of a sharp division betwe...,The merits of Politicizing Domesticity are not...,But the rhetoric of originality in the book’s ...,17,political,reading,...,<em>Laura Lunger Knoppers</em> Politicizing Do...,111,1430,[Arts - Art history],,,2014,2010,jj,ii
17406,http://www.jstor.org/stable/23263669,8,4,To treat Upon Appleton House in these terms is...,MARVELL AND THE STRATEGIC IMAGINATION 377 use ...,To treat Upon Appleton House in these terms is...,,20,political,reading,...,THE REVIEW OF ENGLISH STUDIES PRIZE ESSAY MARV...,63,9049,[Arts - Literature],While recent scholarship has done much to illu...,,2012,2010,jj,ii
48387,http://www.jstor.org/stable/26158923,16,24,Fumaroli's political __reading__ of La Fontain...,4.\nIn dissociating Pound's poetry from his po...,Fumaroli's political reading of La Fontaine's ...,7.\nThe range of adaptations includes critique...,2,political,reading,...,Marianne Moore's Postwar Fables and the Politi...,131,11957,[Arts - Literature],Marianne Moore's response to World War II took...,,2016,2010,jj,ii
15152,http://www.jstor.org/stable/26550046,3,4,"Nevertheless, as is the case with almost every...",Book Review E81poets” (1).\nIn his discussion ...,"Nevertheless, as is the case with almost every...",What he really likes is the playful textuality...,26,political,reading,...,Review Article,114,1050,[Arts - Literature],,,2016,2010,jj,nn1


In [525]:
df_adjs_sample.to_excel('df_adjs_sample.xlsx')

In [479]:
def get_eg_str(word_decade_df, max_sents_per_jrnl=1, max_len=2000):
    # egdf = word_decade_df if len(word_decade_df) <= max_sents else word_decade_df.sample(max_sents)
    # egdf = egdf.drop_duplicates(subset=['url','sent'])
    egdf = word_decade_df.copy()
    word = egdf.token0.iloc[0]
    egs = []
    jrnl_counts = word_decade_df['isPartOf'].value_counts()
    egdf['jrnl_count'] = egdf['isPartOf'].map(jrnl_counts)
    egdf = egdf.sort_values(['jrnl_count', 'isPartOf'], ascending=[False, True]).drop(columns=['jrnl_count'])
    jrnl_groups = list(egdf.groupby('isPartOf'))
    jrnl_groups.sort(key=lambda x: jrnl_counts[x[0]], reverse=True)
    for jrnl, jrnl_df in jrnl_groups:
        jrnl_count = jrnl_counts[jrnl]
        jrnl_df = jrnl_df.sample(frac=1).iloc[:max_sents_per_jrnl]
        egs.append(f'{jrnl.upper()} (n={jrnl_count})')
        for grp, grpdf in jrnl_df.drop_duplicates(subset=['sent']).groupby('url'):
            row = grpdf.iloc[0]
            author = row.creator[0].title() if row.creator else "[Unknown]"
            if len(row.creator) > 1:
                author += " et al."
            title = row.title
            year = row.year
            journal = row.isPartOf
            vol = row.volumeNumber
            issue = row.issueNumber
            # this_eg = [f'##### {author}, "{title}", {vol}.{issue} ({year})']
            for sent in grpdf.sent.tolist():
                egs.append(f'> “{refmt_sent_with_prev_token(sent, word)}” ({author} {year})')
        egs.append('')
    out = '\n'.join(egs)
    if len(out) > max_len:
        out = out[:max_len].strip() + '...'
    return out

def refmt_sent_with_prev_token(sent, word):
    import re
    # Match: word + space + __marked_text__
    # Replace: __word marked_text__
    return re.sub(r'(\S+)\s+__(.*?)__', lambda m: f'{m.group(1).upper()} {m.group(2).upper()}' if m.group(1)==word else f'{m.group(1)} {m.group(2)}', sent)
    


def get_decade_counts(df, max_rank=100):
    new_ld = []
    for decade, decade_df in df.groupby('decade'):
        tok_counts = decade_df.token0.value_counts()
        tok_rel_freq = tok_counts / tok_counts.sum()
        
        for rank, (tok, rel_freq) in enumerate(tok_rel_freq.items()):
            if rank >= max_rank:
                break

            word_dec_df = decade_df[decade_df.token0 == tok]

            import numpy as np
            if len(word_dec_df) > 0:
                weighted_avg_year = int(word_dec_df.loc[word_dec_df['freq'].idxmax(), 'year']) if 'freq' in word_dec_df and not word_dec_df.empty else None
                # freq = word_dec_df['freq'] if 'freq' in word_dec_df else np.ones(len(word_dec_df))
                # weighted_avg_year = int(round(np.average(word_dec_df['year'].astype(float), weights=freq)))
            else:
                weighted_avg_year = None
            egs = get_eg_str(word_dec_df)
            sents = word_dec_df.sent.tolist()

            total_sum = len(word_dec_df)
            total_dec = len(decade_df)
            total_tok = len(df[df.token0 == tok])
            # word_max_freq = df[df.token0 == tok]['freq'].max()
            # word_avg_freq = df[df.token0 == tok]['freq'].mean()
            # word_sum_freq = df[df.token0 == tok]['freq'].sum()
            

            decade_d = {
                'decade': decade,
                'token': tok,
                'rank': rank+1,
                'count': tok_counts[tok],
                'freq': rel_freq,
                # 'word_freq_max': word_max_freq,
                # 'word_freq_avg': word_avg_freq,
                # 'word_freq_sum': word_sum_freq,
                # 'word_count_sum': total_sum,
                # 'word_avg_year': weighted_avg_year,
                "egs": egs,
                **(word2data[tok] if tok in word2data else {}),
            }
            new_ld.append(decade_d)
    new_df = pd.DataFrame(new_ld)
    return new_df

In [480]:
decade_word_counts = get_decade_counts(total_df_adjs)
# for eg in decade_word_counts.egs.unique():
#     pass
# print(eg)

In [481]:
eg = decade_word_counts.query('token == "close" & decade == 2010').egs.iloc[0]
print(eg)


THE MODERN LANGUAGE REVIEW (n=368)
> “He concentrates mainly on the tradition of Thomas Mann scholarship focused tightly on Mann's works and a CLOSE READING of the texts.” (Steve Dowden 2010)

PMLA (n=126)
> “Most Spanish literature departments in the United States confirmed these reserva tions and were largely dedicated to a rarefied, effete, and purely empirical study of literature as belles lettres that seldom transcended what British literary critics labeled CLOSE READING: providing a careful and sustained but fairly conventional interpretation of a brief passage from a text while ignoring the social context and the political atmosphere in which the text was embedded.” (Arturo Arias 2016)

THE REVIEW OF ENGLISH STUDIES (n=126)
> “Holmes's primary method is CLOSE READING.” (Janine Rogers 2010)

NEW LITERARY HISTORY (n=108)
> “Critics are debating the merits of CLOSE READING versus distant reading, surface reading versus deep reading, and reading suspiciously versus reading from a mo

In [499]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def get_word_stats(df):
    word_counts = df.groupby('token')['count'].sum()
    word_freqsum = df.groupby('token')['freq'].sum()
    word_freqavg = df.groupby('token')['freq'].mean()
    word_freqmax = df.groupby('token')['freq'].max()
    word_freqmin = df.groupby('token')['freq'].min()
    word_freqlast = df.sort_values('decade', ascending=False).groupby('token')['freq'].first()
    word_freqfirst = df.sort_values('decade', ascending=True).groupby('token')['freq'].first()
    word_decade_max = df.sort_values('freq', ascending=False).groupby('token').decade.first()
    word_decade_min = df.sort_values('freq', ascending=False).groupby('token').decade.last()
    token2historical_avg = df.groupby('token').apply(
        lambda g: np.average(g['decade'], weights=g['freq'] if 'freq' in g else np.ones(len(g)))
    )

    # get ratio from max to min
    word_freq_ratio = word_freqmax / word_freqmin

    word_ld =[]
    for word in token2historical_avg.index:
        this_word_count = word_counts[word]
        this_word_freq_sum = word_freqsum[word]
        this_word_freq_avg = word_freqavg[word]
        this_word_freq_max = word_freqmax[word]
        this_word_freq_min = word_freqmin[word]

        this_word_freq_last = word_freqlast[word]
        this_word_freq_first = word_freqfirst[word]
        this_word_freq_ratio = this_word_freq_max / this_word_freq_min
        this_word_freq_ratio_early_late = this_word_freq_last / this_word_freq_first
        this_word_freq_ratio_early_late_log = np.log(this_word_freq_ratio_early_late)
        this_word_decade_min = word_decade_min[word]
        this_word_decade_max = word_decade_max[word]
        this_historical_avg = int(token2historical_avg[word])
        decade_range = f"{this_word_decade_min}{this_word_decade_max}"
        label = f"“{word} reading”\n[{this_word_decade_max}s ({this_word_freq_max*100:.1f}%) =\n~{this_word_freq_ratio:.0f}x of {this_word_decade_min}s ({this_word_freq_min*100:.1f}%)]"
        word_d = {
            "token": word,
            "count_sum": this_word_count,
            "freq_sum": this_word_freq_sum,
            "freq_avg": this_word_freq_avg,
            "freq_max": this_word_freq_max,
            "freq_min": this_word_freq_min,
            "freq_last": this_word_freq_last,
            "freq_first": this_word_freq_first,
            "freq_ratio": this_word_freq_ratio,
            "freq_ratio_early_late": this_word_freq_ratio_early_late,
            "freq_ratio_early_late_log": this_word_freq_ratio_early_late_log,
            "decade_min": this_word_decade_min,
            "decade_max": this_word_decade_max,
            "historical_avg": this_historical_avg   ,
            "label": label,
        }
        word_ld.append(word_d)
    word_df = pd.DataFrame(word_ld)
    word_df['freq'] = word_df['freq_sum']
    word_df['fpk_max'] = word_df['freq_max'] * 1000
    word_df['historical_avg_decade'] = word_df['historical_avg'] // 10 * 10
    return word_df


In [500]:
word_stats = get_word_stats(decade_word_counts.query('decade>=1920'))
word_stats

,token,count_sum,freq_sum,freq_avg,freq_max,freq_min,freq_last,freq_first,freq_ratio,freq_ratio_early_late,freq_ratio_early_late_log,decade_min,decade_max,historical_avg,label,freq,fpk_max,historical_avg_decade
0,able,2,0.002144,0.002144,0.002144,0.002144,0.002144,0.002144,1.000000,1.000000,0.000000,1950,1950,1950,“able reading”\n[1950s (0.2%) =\n~1x of 1950s ...,0.002144,2.143623,1950
1,accessible,2,0.002833,0.002833,0.002833,0.002833,0.002833,0.002833,1.000000,1.000000,0.000000,1930,1930,1929,“accessible reading”\n[1930s (0.3%) =\n~1x of ...,0.002833,2.832861,1920
2,accurate,21,0.018850,0.004713,0.007082,0.002377,0.002960,0.007082,2.979226,0.417928,-0.872446,1960,1930,1946,“accurate reading”\n[1930s (0.7%) =\n~3x of 19...,0.018850,7.082153,1940
3,active,11,0.002213,0.002213,0.002213,0.002213,0.002213,0.002213,1.000000,1.000000,0.000000,2000,2000,2000,“active reading”\n[2000s (0.2%) =\n~1x of 2000...,0.002213,2.212834,2000
4,actual,74,0.051933,0.006492,0.017778,0.002213,0.002213,0.017778,8.033939,0.124472,-2.083675,2000,1920,1944,“actual reading”\n[1920s (1.8%) =\n~8x of 2000...,0.051933,17.777778,1940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,wise,2,0.002981,0.002981,0.002981,0.002981,0.002981,0.002981,1.000000,1.000000,0.000000,1940,1940,1940,“wise reading”\n[1940s (0.3%) =\n~1x of 1940s ...,0.002981,2.980626,1940
348,worse,2,0.002833,0.002833,0.002833,0.002833,0.002833,0.002833,1.000000,1.000000,0.000000,1930,1930,1929,“worse reading”\n[1930s (0.3%) =\n~1x of 1930s...,0.002833,2.832861,1920
349,worthless,2,0.002833,0.002833,0.002833,0.002833,0.002833,0.002833,1.000000,1.000000,0.000000,1930,1930,1929,“worthless reading”\n[1930s (0.3%) =\n~1x of 1...,0.002833,2.832861,1920
350,wrong,19,0.029035,0.007259,0.014903,0.003215,0.003215,0.006667,4.634873,0.482315,-0.729158,1950,1940,1935,“wrong reading”\n[1940s (1.5%) =\n~5x of 1950s...,0.029035,14.903130,1930


In [501]:
decade_word_counts_with_word_stats = decade_word_counts.merge(word_stats, on='token', how='left', suffixes=('', '_word'))
dict(decade_word_counts_with_word_stats.iloc[0])

{'decade': np.int64(1900),
 'token': 'true',
 'rank': np.int64(460),
 'count': np.int64(17),
 'freq': np.float64(0.09392265193370165),
 'egs': "THE MODERN LANGUAGE REVIEW (n=13)\n> “Here the source, Grimeston, enables us to restore the TRUE READING.” (T. M. Parrott 1908)\n\nMODERN PHILOLOGY (n=2)\n> “But the TRUE READING, I have no doubt, in all three cases, is ginra.” (Fr. Klaeber 1904)\n\nPMLA (n=2)\n> “Both of these implications are inconsistent with a TRUE READING of Dryden's character.” (Wm. E. Bohn 1907)\n",
 'US_or_UK': '  ',
 'fpm_BNC': np.float64(175.27),
 'fpm_COCA': np.float64(194.64),
 'fpm_COHA_1800s': np.float64(398.39),
 'fpm_COHA_1900-49': np.float64(261.88),
 'fpm_COHA_1950-89': np.float64(211.78),
 'fpm_SOAP': np.float64(349.75),
 'fpm_bnc_acad': np.float64(250.01),
 'fpm_bnc_fic': np.float64(205.67),
 'fpm_bnc_mag': np.float64(150.79),
 'fpm_bnc_misc': np.float64(179.79),
 'fpm_bnc_news': np.float64(108.82),
 'fpm_bnc_noAc': np.float64(158.83),
 'fpm_bnc_spok': np.fl

In [502]:
def periodize(year):
    if 1900 <= year < 1945:
        return '1900–1945'
    elif 1945 <= year < 1990:
        return '1945–1990'
    elif 1990 <= year:
        return '1990–'
    else:
        return 'Invalid year'

decade_word_counts_with_word_stats['period'] = decade_word_counts_with_word_stats.decade_max.map(periodize)


filter_n = 20
filter_words = {
    w
    for g,grpdf in decade_word_counts_with_word_stats.groupby('period')
    for w in grpdf.sort_values('count', ascending=False).token[:filter_n]
}

decade_word_counts_with_word_stats['filtered_token'] = decade_word_counts_with_word_stats.token.isin(filter_words)

# filtered_decade_word_counts_with_word_stats = decade_word_counts_with_word_stats[decade_word_counts_with_word_stats.filtered_token]


In [503]:
lbldf = decade_word_counts_with_word_stats[decade_word_counts_with_word_stats.filtered_token].sort_values('freq', ascending=False)
lbldf = lbldf.drop_duplicates(subset=['token'])

with open('data/word_stats_labels.txt', 'w') as f:
    for lbl in lbldf.label.fillna('').tolist():
        f.write(lbl + '\n\n')

# for lbl in word_stats.sort_values('freq_ratio', ascending=False).label.tolist():
#     print(lbl)
#     print()

# print('\n\n'.join(lbldf.label.tolist()))
# lbldf

In [504]:
decade_word_counts_with_word_stats.to_excel('decade_word_counts_with_word_stats.xlsx')

In [505]:
# decade_word_counts_with_word_stats.egs.value_counts()

In [478]:
row=decade_word_counts.sample(1).iloc[0]
printm(f'### "{row.token} reading" in the {row.decade}s')
printm(row.egs)

### "exciting reading" in the 1940s

THE MODERN LANGUAGE REVIEW (n=3)
> “That the work is the result of careful and scholarly research, however, there can be no doubt: though, on the other hand, it cannot be said to afford EXCITING READING, and one could have wished for clearer pictures of the intellectual conditions and personalities involved.” (Elizabeth J. Sweeting 1943)

THE REVIEW OF ENGLISH STUDIES (n=1)
> “The story of the exercise of this right, here reconstructed from the archives of the University and the records of the Stationers' Company, is absorbing, even EXCITING, READING.” (Laurence Hanson 1947)


In [398]:
decade_word_counts.to_excel('decade_word_counts.xlsx')

In [385]:
# !pip install plotnine
import plotnine as p9
p9.options.figure_size = (10, 6)
p9.options.dpi = 300

In [386]:
# !pip install numpy

In [294]:
most_ever_top_words = new_df.groupby('token')['count'].sum().sort_values(ascending=False).index.tolist()
print("distant" in most_ever_top_words)


fig_num_words = 50
fig_top_words = most_ever_top_words[:fig_num_words]
print("distant" in fig_top_words)

ever_top_words_sorted = token2historical_avg.sort_values().index.tolist()
# ever_top_words_sorted = fig_top_words
fig_df = new_df.query('token in @fig_top_words').copy()
fig_df['fpk'] = fig_df['freq'] * 1000
fig_df['historical_avg'] = fig_df['token'].map(token2historical_avg)

token_labels = [
    f"{token} ({int(token2historical_avg[token])})"
    for token in ever_top_words_sorted
    if token in fig_top_words
]

token_labels


fig_df['token_label'] = fig_df.apply(lambda row: f"{row['token']} ({int(row['historical_avg'])})", axis=1)

fig_df['token'] = pd.Categorical(fig_df['token'], categories=ever_top_words_sorted, ordered=True)
fig_df['token_label'] = pd.Categorical(fig_df['token_label'], categories=token_labels, ordered=True)

NameError: name 'new_df' is not defined

In [ ]:
fig = p9.ggplot(fig_df, p9.aes(x="decade", y="fpk", group="token"))
fig += p9.geom_line(size=.5, alpha=.15, color="blue")
fig += p9.geom_point(p9.aes(size="count"), alpha=.15, color="blue")
fig += p9.geom_text(p9.aes(label="count"), size=8, angle=45)
fig += p9.facet_wrap("token_label", ncol=10)
fig += p9.theme_classic()
fig += p9.theme(figure_size=(16, 9))
fig


In [ ]:
word_counts = new_df.groupby('token')['count'].sum()
word_freqsum = new_df.groupby('token')['freq'].sum()
word_freqavg = new_df.groupby('token')['freq'].mean()
word_freqmax = new_df.groupby('token')['freq'].max()
word_ld =[]
for word in token2historical_avg.index:
    word_d = {
        "token": word,
        "count_sum": word_counts[word],
        "freq_sum": word_freqsum[word],
        "freq_avg": word_freqavg[word],
        "freq_max": word_freqmax[word],
        "historical_avg": int(token2historical_avg[word]),
    }
    word_ld.append(word_d)
word_df = pd.DataFrame(word_ld)
word_df['freq'] = word_df['freq_sum']
word_df['fpk_max'] = word_df['freq_max'] * 1000
word_df


In [ ]:
word_fig_df = word_df[word_df.count_sum > 10]
# print("distant" in word_fig_df.token.tolist())
fig = p9.ggplot(word_fig_df, p9.aes(x="historical_avg", y="fpk_max"))
fig += p9.geom_text(p9.aes(label="token"), size=12)
fig += p9.theme_minimal()
fig += p9.theme(figure_size=(16, 9))
fig += p9.scale_y_log10()
fig


In [ ]:
word_df.to_csv('word_stats.csv')

In [ ]:
total_df_adjs.decade #.query('token0 == "distant"').sent.tolist()

In [ ]:
total_df_adjs

In [ ]:
grp,grpdf = next((g,grpdf) for g,grpdf in total_df_adjs.groupby(['decade','token0']) if g[1] == "distant")
grpdf

In [ ]:
dict(grpdf.iloc[0])

In [ ]:
# dict(total_df_adjs.iloc[0])

In [ ]:
def get_eg_str(word_decade_df, max_sents=10):
    egdf = word_decade_df if len(word_decade_df) <= max_sents else word_decade_df.sample(max_sents)

    egs = []
    for grp, grpdf in egdf.groupby('url'):
        row = grpdf.iloc[0]
        author = row.creator
        if len(row.creator) > 1:
            author += " et al."
        title = row.title
        year = row.year
        journal = row.isPartOf
        vol = row.volumeNumber
        issue = row.issueNumber
        this_eg = [f'{author}, "{title}", _{journal}_, vol. {vol}, no. {issue} ({year})']
        for sent in grpdf.sent.tolist():
            this_eg.append(f'  * {sent}')
        egs.append("\n".join(this_eg))
    return '\n\n'.join(egs)



    
    

In [ ]:
# total_df_adjs

In [ ]:
word_decade_to_egs = {
    (decade, word): get_eg_str(word_decade_df)
    for (decade, word), word_decade_df in total_df_adjs.groupby(['decade','token0'])
}

out_df = new_df.copy()
out_df['egs'] = out_df.apply(lambda row: word_decade_to_egs[(row.decade, row.token)], axis=1)

out_df = out_df.merge(word_df, on='token', how='left', suffixes=('', '_word'))




In [ ]:
out_df

In [ ]:
# !pip install openpyxl
out_df.to_excel('word_decade_stats.xlsx', index=False)